In [1]:
# class imports from QMSC
from qmsc.ansatz import FlexibleAnsatz

# Building an ansatz with FlexibleAnsatz
* A `FlexbileAnsatz` object is instantiated with the # of qubits ansatz is over, i.e.
`ansatz = FlexibleAnsatz(2)`
    * class member data `ansatz.cwa` contains ansatz information in the form `[('name1', param_list1, qubit_list1), ('name2', param_list2, qubit_list2)]`
* Basic way to add elements is with `insert_x` methods where `x` is an ansatz type
    * Takes as argument qubits to build `x` over
    * Optinally takes `loc` as location to place elements

In [2]:
ns = 1
na = 1
ansatz = FlexibleAnsatz(ns + na)
ansatz.insert_givens_layers([0, 1])
print("Printing ansatz.cwa")
print(ansatz.cwa)
print("===================================")
print("Default print returns in more readable new-line delimited string.")
print(ansatz)

Printing ansatz.cwa
[('G', [0], [0, 1]), ('G2', [0], [0, 1]), ('G', [0], [0, 1])]
Default print returns in more readable new-line delimited string.
G([0])_([0, 1])
G2([0])_([0, 1])
G([0])_([0, 1])



## The `FlexibleAnsatz` class has many useful methods `get` used below

In [3]:
ansatz.get_insertion_locations()

[0, 1, 2, 3]

In [4]:
ansatz.get_instructions()

['G', 'G2', 'G']

In [5]:
ansatz.get_num_instructions()

3

In [6]:
ansatz.get_num_parameters()

3

In [7]:
ansatz.get_instructions()

['G', 'G2', 'G']

## We can insert a new gate motif--say an arbitrary one qubit gate via Euler decomposition on qubit 1--in the middle of the above motif and then initialize some random parameters
* Editorial note: That this method takes as argument an integer but before the `insert_givens_layer` used a list is a testament to this not being "production-level" code. Sorry!
* We could say the same about the default behavior: For Givens, it defaulted to making angles 0--here it defaulted to random angles.

In [8]:
ansatz.insert_arb1q(1, loc=1)
print(ansatz)

G([0])_([0, 1])
rz([0.39841959800056326])_([1])
ry([1.163376283479179])_([1])
rz([3.5358645708979624])_([1])
G2([0])_([0, 1])
G([0])_([0, 1])



In [9]:
ansatz.update_parameters([0, 1, 2, 3, 4, 5])
print(ansatz)

G([0])_([0, 1])
rz([1])_([1])
ry([2])_([1])
rz([3])_([1])
G2([4])_([0, 1])
G([5])_([0, 1])



In [10]:
ansatz.resample_parameters()
print(ansatz)

G([2.5446847062526103])_([0, 1])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])



## Convert to a circuit for `qiskit` style control

In [11]:
ansatz_circ = ansatz.build_circ(2)
print("Printing circuit in default `abstract form`")
print(ansatz_circ.draw())
print("===================================")
print("Printing circuit after unraveling abstracted gates.")
print(ansatz_circ.decompose().draw())

Printing circuit in default `abstract form`
     ┌──────────┐                                          ┌──────────┐»
q_0: ┤0         ├──────────────────────────────────────────┤0         ├»
     │  Unitary │┌────────────┐┌────────────┐┌────────────┐│  Unitary │»
q_1: ┤1         ├┤ Rz(5.2247) ├┤ Ry(5.1867) ├┤ Rz(3.4685) ├┤1         ├»
     └──────────┘└────────────┘└────────────┘└────────────┘└──────────┘»
«     ┌──────────┐
«q_0: ┤0         ├
«     │  Unitary │
«q_1: ┤1         ├
«     └──────────┘
Printing circuit after unraveling abstracted gates.
global phase: 4.385
        ┌───────────────────┐            ┌────────────────────┐         »
q_0: ───┤ U3(π/2,-π/2,3π/8) ├─────■──────┤ U3(1.8693,-π,-π/2) ├──────■──»
     ┌──┴───────────────────┴──┐┌─┴─┐┌───┴────────────────────┴───┐┌─┴─┐»
q_1: ┤ U3(0.0013233,-π/2,-π/8) ├┤ X ├┤ U3(1.5717,0.29846,-1.5695) ├┤ X ├»
     └─────────────────────────┘└───┘└────────────────────────────┘└───┘»
«        ┌─────────────────┐   ┌─────────────────┐    

#### We can also choose to change ordering of qubits from ansatz to circuit--this can help deal with native qubit topology on a device compared to ansatz

In [12]:
ansatz_circ = ansatz.build_circ(2, qubit_mapping = {0: 1, 1:0})
print(ansatz_circ.draw())

     ┌──────────┐┌────────────┐┌────────────┐┌────────────┐┌──────────┐»
q_0: ┤1         ├┤ Rz(5.2247) ├┤ Ry(5.1867) ├┤ Rz(3.4685) ├┤1         ├»
     │  Unitary │└────────────┘└────────────┘└────────────┘│  Unitary │»
q_1: ┤0         ├──────────────────────────────────────────┤0         ├»
     └──────────┘                                          └──────────┘»
«     ┌──────────┐
«q_0: ┤1         ├
«     │  Unitary │
«q_1: ┤0         ├
«     └──────────┘


## More advanced feature 1: Adding an identity perturbation

In [13]:
print(ansatz)
ansatz.insert_random_idres()
print("===================================")
print("printed ansatz after adding identity perturbation")
print(ansatz)

G([2.5446847062526103])_([0, 1])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])

printed ansatz after adding identity perturbation
G([2.5446847062526103])_([0, 1])
cnot(None)_([1, 0])
rz([0])_([1])
rx([0])_([1])
rz([0])_([1])
rx([0])_([0])
rz([0])_([0])
rx([0])_([0])
cnot(None)_([1, 0])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])



## More advanced feature 2: Removing contiguous gates which are collectively identity

In [14]:
print(ansatz)
ansatz.remove_identity_elements()
print("===================================")
print("printed ansatz after removing identity perturbation")
print(ansatz)

G([2.5446847062526103])_([0, 1])
cnot(None)_([1, 0])
rz([0])_([1])
rx([0])_([1])
rz([0])_([1])
rx([0])_([0])
rz([0])_([0])
rx([0])_([0])
cnot(None)_([1, 0])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])

printed ansatz after removing identity perturbation
G([2.5446847062526103])_([0, 1])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])



## More advanced feature 3: Adding an identity perturbation before/after instruction

In [17]:
print(ansatz)
ansatz.insert_random_idres_after_loc(after_loc=3)
print("===================================")
print("printed ansatz after adding identity perturbation")
print(ansatz)

G([2.5446847062526103])_([0, 1])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])

printed ansatz after adding identity perturbation
G([2.5446847062526103])_([0, 1])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
cnot(None)_([1, 0])
rz([0])_([1])
rx([0])_([1])
rz([0])_([1])
rx([0])_([0])
rz([0])_([0])
rx([0])_([0])
cnot(None)_([1, 0])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])



In [18]:
print(ansatz)
ansatz.insert_random_idres_before_loc(before_loc=2)
print("===================================")
print("printed ansatz after adding identity perturbation")
print(ansatz)

G([2.5446847062526103])_([0, 1])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
cnot(None)_([1, 0])
rz([0])_([1])
rx([0])_([1])
rz([0])_([1])
rx([0])_([0])
rz([0])_([0])
rx([0])_([0])
cnot(None)_([1, 0])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])

printed ansatz after adding identity perturbation
rz([0])_([1])
ry([0])_([1])
rz([0])_([1])
rz([0])_([0])
ry([0])_([0])
rz([0])_([0])
G([2.5446847062526103])_([0, 1])
rz([5.224698257232423])_([1])
ry([5.186739502800468])_([1])
cnot(None)_([1, 0])
rz([0])_([1])
rx([0])_([1])
rz([0])_([1])
rx([0])_([0])
rz([0])_([0])
rx([0])_([0])
cnot(None)_([1, 0])
rz([3.4684616095443426])_([1])
G2([0.7914025654961377])_([0, 1])
G([4.532332766967361])_([0, 1])

